### Import Dependencies

In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sns
%matplotlib inline
import pulp

In [4]:
import warnings
warnings.filterwarnings('ignore')

### Load Dataset

In [24]:
df = pd.read_csv('nutrition.csv').drop('Unnamed: 0',axis=1)

### EDA

In [27]:
df.head()

,name,serving_size,calories,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,...,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
0,Cornstarch,100 g,381,0.1g,NaN,0,9.00 mg,0.4 mg,0.00 mcg,0.00 mcg,...,0.05 g,0.009 g,0.016 g,0.025 g,0.00 mg,0.0 g,0.09 g,0.00 mg,0.00 mg,8.32 g
1,"Nuts, pecans",100 g,691,72g,6.2g,0,0.00 mg,40.5 mg,22.00 mcg,0.00 mcg,...,71.97 g,6.180 g,40.801 g,21.614 g,0.00 mg,0.0 g,1.49 g,0.00 mg,0.00 mg,3.52 g
2,"Eggplant, raw",100 g,25,0.2g,NaN,0,2.00 mg,6.9 mg,22.00 mcg,0.00 mcg,...,0.18 g,0.034 g,0.016 g,0.076 g,0.00 mg,0.0 g,0.66 g,0.00 mg,0.00 mg,92.30 g
3,"Teff, uncooked",100 g,367,2.4g,0.4g,0,12.00 mg,13.1 mg,0,0,...,2.38 g,0.449 g,0.589 g,1.071 g,0,0,2.37 g,0,0,8.82 g
4,"Sherbet, orange",100 g,144,2g,1.2g,1mg,46.00 mg,7.7 mg,4.00 mcg,0.00 mcg,...,2.00 g,1.160 g,0.530 g,0.080 g,1.00 mg,0.0 g,0.40 g,0.00 mg,0.00 mg,66.10 g


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8789 entries, 0 to 8788
Data columns (total 76 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   name                         8789 non-null   object
 1   serving_size                 8789 non-null   object
 2   calories                     8789 non-null   int64 
 3   total_fat                    8789 non-null   object
 4   saturated_fat                7199 non-null   object
 5   cholesterol                  8789 non-null   object
 6   sodium                       8789 non-null   object
 7   choline                      8789 non-null   object
 8   folate                       8789 non-null   object
 9   folic_acid                   8789 non-null   object
 10  niacin                       8789 non-null   object
 11  pantothenic_acid             8789 non-null   object
 12  riboflavin                   8789 non-null   object
 13  thiamin                      8789

In [35]:
df.columns

Index(['name', 'serving_size', 'calories', 'total_fat', 'saturated_fat',
       'cholesterol', 'sodium', 'choline', 'folate', 'folic_acid', 'niacin',
       'pantothenic_acid', 'riboflavin', 'thiamin', 'vitamin_a',
       'vitamin_a_rae', 'carotene_alpha', 'carotene_beta',
       'cryptoxanthin_beta', 'lutein_zeaxanthin', 'lucopene', 'vitamin_b12',
       'vitamin_b6', 'vitamin_c', 'vitamin_d', 'vitamin_e', 'tocopherol_alpha',
       'vitamin_k', 'calcium', 'copper', 'irom', 'magnesium', 'manganese',
       'phosphorous', 'potassium', 'selenium', 'zink', 'protein', 'alanine',
       'arginine', 'aspartic_acid', 'cystine', 'glutamic_acid', 'glycine',
       'histidine', 'hydroxyproline', 'isoleucine', 'leucine', 'lysine',
       'methionine', 'phenylalanine', 'proline', 'serine', 'threonine',
       'tryptophan', 'tyrosine', 'valine', 'carbohydrate', 'fiber', 'sugars',
       'fructose', 'galactose', 'glucose', 'lactose', 'maltose', 'sucrose',
       'fat', 'saturated_fatty_acids', 'mon

In [43]:
df = df[['name', 'calories', 'total_fat', 'protein', 'water', 'carbohydrate', 'fiber']]

In [45]:
df.head()

,name,calories,total_fat,protein,water,carbohydrate,fiber
0,Cornstarch,381,0.1g,0.26 g,8.32 g,91.27 g,0.9 g
1,"Nuts, pecans",691,72g,9.17 g,3.52 g,13.86 g,9.6 g
2,"Eggplant, raw",25,0.2g,0.98 g,92.30 g,5.88 g,3.0 g
3,"Teff, uncooked",367,2.4g,13.30 g,8.82 g,73.13 g,8.0 g
4,"Sherbet, orange",144,2g,1.10 g,66.10 g,30.40 g,1.3 g


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8789 entries, 0 to 8788
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          8789 non-null   object
 1   calories      8789 non-null   int64 
 2   total_fat     8789 non-null   object
 3   protein       8789 non-null   object
 4   water         8789 non-null   object
 5   carbohydrate  8789 non-null   object
 6   fiber         8789 non-null   object
dtypes: int64(1), object(6)
memory usage: 480.8+ KB


In [49]:
df.columns

Index(['name', 'calories', 'total_fat', 'protein', 'water', 'carbohydrate',
       'fiber'],
      dtype='object')

In [63]:
df['total_fat'] = df['total_fat'].str.replace('g', '', regex=False)

df['total_fat'] = df['total_fat'].astype('float')

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8789 entries, 0 to 8788
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          8789 non-null   object 
 1   calories      8789 non-null   int64  
 2   total_fat     8789 non-null   float64
 3   protein       8789 non-null   object 
 4   water         8789 non-null   object 
 5   carbohydrate  8789 non-null   object 
 6   fiber         8789 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 480.8+ KB


In [73]:
lists = ['protein', 'water', 'carbohydrate',
       'fiber']

for col in lists:
    df[col] = df[col].str.replace(' g', '', regex=False)
    df[col] = df[col].astype('float')

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8789 entries, 0 to 8788
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          8789 non-null   object 
 1   calories      8789 non-null   int64  
 2   total_fat     8789 non-null   float64
 3   protein       8789 non-null   float64
 4   water         8789 non-null   float64
 5   carbohydrate  8789 non-null   float64
 6   fiber         8789 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 480.8+ KB


In [81]:
df.head()

,name,calories,total_fat,protein,water,carbohydrate,fiber
0,Cornstarch,381,0.1,0.26,8.32,91.27,0.9
1,"Nuts, pecans",691,72.0,9.17,3.52,13.86,9.6
2,"Eggplant, raw",25,0.2,0.98,92.30,5.88,3.0
3,"Teff, uncooked",367,2.4,13.30,8.82,73.13,8.0
4,"Sherbet, orange",144,2.0,1.10,66.10,30.40,1.3


In [85]:
week_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
split_values = np.linspace(0,len(df),8).astype(int)
split_values[-1] = split_values[-1]-1
def random_dataset():
    frac_data = df.sample(frac=1).reset_index().drop('index',axis=1)
    day_data = []
    for s in range(len(split_values)-1):
        day_data.append(frac_data.loc[split_values[s]:split_values[s+1]])
    return dict(zip(week_days,day_data))

In [87]:
random_dataset()['Monday']

,name,calories,total_fat,protein,water,carbohydrate,fiber
0,"Waffle, microwave, ready-to-heat, frozen, plain",298,9.9,6.71,34.95,45.41,2.4
1,"Apples, heated, unsweetened, frozen",47,0.3,0.29,87.16,12.00,1.3
2,"Spices, dried, tarragon",295,7.2,22.77,7.74,50.22,7.4
3,MORNINGSTAR FARMS Spicy Black Bean Enchilada E...,123,5.2,5.80,71.00,15.90,2.6
4,"Chicken, raw, all classes, heart",153,9.3,15.55,73.56,0.71,0.0
...,...,...,...,...,...,...,...
1251,"Cake, prepared from recipe, pineapple upside-down",319,12.0,3.50,32.30,50.50,0.8
1252,"KELLOGG'S, Blueberry, Waffles, NUTRI-GRAIN, EGGO",257,7.8,6.20,39.25,43.60,3.9
1253,"Fast foods, with sausage, biscuit",371,24.0,9.67,32.82,29.99,0.4
1254,"Mushrooms, without salt, cooked, shiitake",56,0.2,1.56,83.48,14.39,2.1


In [89]:
def build_nutritional_values(kg,calories):
    protein_calories = kg*4
    res_calories = calories-protein_calories
    carb_calories = calories/2.
    fat_calories = calories-carb_calories-protein_calories
    res = {'Protein Calories':protein_calories,'Carbohydrates Calories':carb_calories,'Fat Calories':fat_calories}
    return res

In [91]:
def extract_gram(table):
    protein_grams = table['Protein Calories']/4.
    carbs_grams = table['Carbohydrates Calories']/4.
    fat_grams = table['Fat Calories']/9.
    res = {'Protein Grams':protein_grams, 'Carbohydrates Grams':carbs_grams,'Fat Grams':fat_grams}
    return res

In [93]:
build_nutritional_values(70,2000)

{'Protein Calories': 280,
 'Carbohydrates Calories': 1000.0,
 'Fat Calories': 720.0}

In [95]:
print(extract_gram(build_nutritional_values(70,2000)))

{'Protein Grams': 70.0, 'Carbohydrates Grams': 250.0, 'Fat Grams': 80.0}


In [111]:
def model(prob,day,kg,calories):
    G = extract_gram(build_nutritional_values(kg,calories))
    E = G['Carbohydrates Grams']
    F = G['Fat Grams']
    P = G['Protein Grams']
    day_data = days_data[day]
    day_data = day_data[day_data.calories!=0]
    food = day_data.name.tolist()
    c  = day_data.calories.tolist()
    x  = pulp.LpVariable.dicts( "x", indices = food, lowBound=0, upBound=1.5, cat='Continuous', indexStart=[] )
    e = day_data.carbohydrate.tolist()
    f = day_data.total_fat.tolist()
    p = day_data.protein.tolist()
#    prob  = pulp.LpProblem( "Diet", LpMinimize )
    prob += pulp.lpSum( [x[food[i]]*c[i] for i in range(len(food))]  )
    prob += pulp.lpSum( [x[food[i]]*e[i] for i in range(len(x)) ] )>=E
    prob += pulp.lpSum( [x[food[i]]*f[i] for i in range(len(x)) ] )>=F
    prob += pulp.lpSum( [x[food[i]]*p[i] for i in range(len(x)) ] )>=P
    prob.solve()
    variables = []
    values = []
    for v in prob.variables():
        variable = v.name
        value = v.varValue
        variables.append(variable)
        values.append(value)
    values = np.array(values).round(2).astype(float)
    sol = pd.DataFrame(np.array([food,values]).T, columns = ['Food','Quantity'])
    sol['Quantity'] = sol.Quantity.astype(float)
    sol = sol[sol['Quantity']!=0.0]
    sol.Quantity = sol.Quantity*100
    sol = sol.rename(columns={'Quantity':'Quantity (g)'})
    return sol

In [133]:
sol_monday = model('Monday',70,1500, 4000)

KeyError: 70

In [129]:
diet = total_model(70,3000)

Building a model for day Monday 



PulpSolverError: Pulp: Error while executing C:\Users\Asus\anaconda3\Lib\site-packages\pulp\solverdir\cbc\win\64\cbc.exe

In [101]:
!pip install pulp

  Obtaining dependency information for pulp from https://files.pythonhosted.org/packages/09/d7/57e71e11108203039c895643368c0d1a99fe719a6a80184edf240c33d25f/PuLP-2.8.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB 653.6 kB/s eta 0:00:27
   ---------------------------------------- 0.1/17.7 MB 1.2 MB/s eta 0:00:15
   ---------------------------------------- 0.1/17.7 MB 1.2 MB/s eta 0:00:15
    --------------------------------------- 0.3/17.7 MB 1.4 MB/s eta 0:00:13
    --------------------------------------- 0.3/17.7 MB 1.4 MB/s eta 0:00:13
    --------------------------------------- 0.4/17.7 MB 1.5 MB/s eta 0:00:12
    --------------------------------------- 0.4/17.7 MB 1.5 MB/s eta 0:00:12
   - -------------------------------------- 0.5/17.7 MB 1.4 MB/s eta 0:00:13
   - -------------------------------------- 0.7/1